#Boston 311 v9 - Using Our Code as a Python Package

This codebase is far from mature, but it has become unwieldy to manage in one Jupyter notebook. Therefore I have created a github repository and structured the code as a python package that can be installed with pip.

This will greatly simplify our Jupyter notebooks going forward.

I will probably still use Jupyter notebooks to develop the code so I can keep track of my thought process and make it easier to create presentations. The github repository will always have the most up to date tools and code.

##Questions and To-Dos:

Below is our open questions and to-dos consolidated from the last notebook. Moving forward we will probably keep this list at the top of each notebook.

2. Add more features
3. clean up the data by removing outliers
6. look at the currently available android app and see what values are available to the user to select, and which categories might be assigned by the 311 agents after receiving a new case.
7. compare a basic model which only uses the department value as a feature to our more complex models as a heuristic for whether additional features actually improve predictions.
8. Moving forward compare our model predictions with the target date assigned by 311 to see which performs better.

Questions to answer:
1. Can we find some basic commonality between open cases?
2. When and how is the target date set? How about the overdue flag?
3. Do cases autoclose after a certain time?

###Conclusions from this notebook, copied from below:
We have a problem now though, which is that our data cleaning functions drop the case_enquiry_id before returning the data, which is good for training, but it means we can't match up prediction results with the original cases. 

Our immediate next task for prediction should be to refactor the data cleaning and splitting functions to make predicting cases possible with a particular model.

###Conclusions from Boston311_v8:

We got some variable results on these models. Now that we have several scenarios, we might want to come up with ways to compare the performance of these models easily.

We are reaching the limits of the capabilities of Google Colaboratory. When we train our models, we might want to try deleting the test data after splitting it so we can save ram. If we want to save it we can write it to a file and download it before deleting it.

Additionally, we want to delete any intermediary data frames created during training before doing the next training. The best way to do that will be to put our data splitting and training inside functions so when the functions complete the variables go out of scope and the RAM they used is freed. Anything that needs to be kept can be saved to files.


##Install the package from github using pip

In [2]:
! pip install git+https://github.com/mindfulcoder49/Boston_311.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mindfulcoder49/Boston_311.git to /tmp/pip-req-build-dhrown4p
  Running command git clone --filter=blob:none --quiet https://github.com/mindfulcoder49/Boston_311.git /tmp/pip-req-build-dhrown4p
  Resolved https://github.com/mindfulcoder49/Boston_311.git to commit 7c49272b6ec167e1cdf47108f672a6451b346cd1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.0.2-py3-none-any.whl size=10759 sha256=f2fd99bf9ebdaae454aef866033a9c46b6807b84ed08c4fd024ebf11bdd9d4da
  Stored in directory: /tmp/pip-ephem-wheel-cache-24j0eo7y/wheels/5d/ab/56/ff57b10a3a91b6b72d020ff3d6d6c867b688c31c2a2143e526
Successfully built boston311


##Load the Data using the boston311 module we just installed

In [1]:
from boston311.load_data import load_data_from_urls
data = load_data_from_urls()

/usr/local/lib/python3.10/dist-packages/boston311/load_data.py:27: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/usr/local/lib/python3.10/dist-packages/boston311/load_data.py:27: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


##Train a Model with a specific scenario

### Basic logistic model

No outlier removal. y output is a series of 0 or 1 corresponding to whether a case is Open or Closed, with 0 marking Open and 1 marking closed. X dataframe should contain only dummied columns for the 'subject', 'reason', 'department', 'source', and 'ward_number' columns.

### Basic linear model

No outlier removal. y output is a series of floats corresponding to the number of hours between case open date and case close date. All open cases are dropped. 


| Scenario | Description |
| --- | --- |
| **Logistic cleaning:** |  |
| Scenario 0 | No Change from Basic |
| Scenario 1 | drop any open cases from the last month |
| Scenario 2 | switch the event value for any cases that took longer than a month to close. |
| Scenario 3 | all records where source is "Employee Generated" or "City Worker App" are removed. |
| Scenario 4 | all records where survival_time is less than an hour are removed. |
| Scenario 5 | the type column is one-hot encoded and added to the data. |
| **Linear cleaning:** |  |
| Scenario 0 | no change from basic |
| Scenario 1 | remove records if the case took more than a month to close |
| Scenario 2 | remove records only if the time to close is negative |
| Scenario 3 | all records where source is "Employee Generated" or "City Worker App" are removed. |
| Scenario 4 | all records where survival_time is less than an hour are removed. |
| Scenario 5 | the type column is one-hot encoded and added to the data. |




In [2]:
from boston311.train_models import train_logistic_model
model = train_logistic_model(data, [1,2,3,4,5])

Starting Training at 2023-05-13 19:06:54.613832
Epoch 1/10
13724/13724 [==============================] - 34s 2ms/step - loss: 0.2835 - accuracy: 0.8983 - val_loss: 0.2489 - val_accuracy: 0.9102
Epoch 2/10
13724/13724 [==============================] - 33s 2ms/step - loss: 0.2473 - accuracy: 0.9103 - val_loss: 0.2457 - val_accuracy: 0.9110
Epoch 3/10
13724/13724 [==============================] - 33s 2ms/step - loss: 0.2457 - accuracy: 0.9108 - val_loss: 0.2447 - val_accuracy: 0.9115
Epoch 4/10
13724/13724 [==============================] - 34s 2ms/step - loss: 0.2452 - accuracy: 0.9109 - val_loss: 0.2444 - val_accuracy: 0.9116
Epoch 5/10
13724/13724 [==============================] - 34s 2ms/step - loss: 0.2449 - accuracy: 0.9109 - val_loss: 0.2443 - val_accuracy: 0.9110
Epoch 6/10
13724/13724 [==============================] - 34s 2ms/step - loss: 0.2448 - accuracy: 0.9109 - val_loss: 0.2441 - val_accuracy: 0.9115
Epoch 7/10
13724/13724 [==============================] - 35s 3ms/step

##Predict the Outcome of Open Cases

The scenario we ran dropped all the cases from the last month, so we should predict the cases from the last month. 

We have a problem now though, which is that our data cleaning functions drop the case_enquiry_id before returning the data, which is good for training, but it means we can't match up prediction results with the original cases. 

Our immediate next task for prediction should be to refactor the data cleaning and splitting functions to make predicting cases possible with a particular model.


##Save the model

In [4]:
from google.colab import files

model.save("model_log_1234.h5")
files.download('model_log_1234.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import gc
gc.collect()

2469